In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np

username = "artia_readonly"
password = 'ap3ljax#!0frack'
host = "artia-dashboard-psql.postgres.database.azure.com"
port = "5432"              
database = "artia_dashboard"

url = URL.create(
    "postgresql+psycopg2",
    username=username,
    password=password,
    host=host,
    port=port,
    database=database,
)

engine = create_engine(url)

In [2]:
with open('user_subscription_report.sql', 'r') as f:
    sql_query = f.read()

df = pd.read_sql(sql_query, engine)
df.to_csv('user_subscription_report.csv', index=False)

In [3]:
with open('coverage_data.sql', 'r') as f:
    sql_query = f.read()
    
df = pd.read_sql(sql_query, engine)
drug_df = df
drug_df['capsule_name'] = [product + ' ' + tag if tag != '' else product for product, tag in zip(drug_df['Product'], drug_df['Coverage Tag'])]
drug_df = drug_df[['ST', 'capsule_name', 'Class']]
drug_df.columns = ['ST','Drugs','Class']
drug_df = drug_df.drop_duplicates().reset_index(drop=True)
drug_df.to_csv('drugs.csv', index=False)

In [72]:
coverage_tags = pd.read_sql("SELECT * FROM coverage_tags", engine)
state_coverages = pd.read_sql("SELECT * FROM state_coverages", engine)
state_coverage_coverage_tags = pd.read_sql("SELECT * FROM state_coverage_coverage_tags", engine)
drugs = pd.read_sql("SELECT * FROM drugs", engine)
states = pd.read_sql("SELECT * FROM states", engine)

In [84]:
coverage_data = pd.DataFrame()

state_name_map = states.set_index('id')['name'].to_dict()
coverage_data['State'] = state_coverages['state_id'].map(state_name_map)

state_map = states.set_index('id')['code'].to_dict()
coverage_data['ST'] = state_coverages['state_id'].map(state_map)

drug_map = drugs.set_index('id')['name'].to_dict()
coverage_data['Product'] = state_coverages['drug_id'].map(drug_map)

coverage_tag_map = coverage_tags.set_index('id')['name'].to_dict()
coverage_tag_array = state_coverage_coverage_tags['coverage_tag_id'].map(coverage_tag_map)

coverage_tag_df = pd.DataFrame()
coverage_tag_df['coverage_tag'] = coverage_tag_array
coverage_tag_df['state_coverage_id'] = state_coverage_coverage_tags['state_coverage_id']

state_coverage_coverage_tag_map = (
    coverage_tag_df
    .groupby('state_coverage_id')['coverage_tag']
    .apply(lambda x: ', '.join(sorted(set(x.dropna()))))
    .to_dict()
)
coverage_data['Coverage Tag'] = state_coverages['id'].map(state_coverage_coverage_tag_map)

coverage_data = coverage_data.sort_values('State').reset_index(drop=True)

coverage_data['Coverage Tag'] = coverage_data['Coverage Tag'].replace(np.nan, '')

coverage_data['capsule_name'] = [product + ' ' + tag if tag != '' else product for product, tag in zip(coverage_data['Product'], coverage_data['Coverage Tag'])]

In [5]:
with open('A_team_export.sql', 'r') as f:
    sql_query = f.read()
    
df = pd.read_sql(sql_query, engine)
df.to_csv('A_team_export.csv', index=False)

In [42]:
drug = 'Vitafol-Ob+Dha'
pdl_df = pd.DataFrame()
pdl_df['pdl_name'] = ['VITAFOL-OB+DHA']

first_word = drug.split()[0].lower()
matches = pdl_df[pdl_df['pdl_name'].str.lower().str.contains(first_word, na=False, regex=False)]

In [43]:
matches

,pdl_name
0,VITAFOL-OB+DHA
